# Day 4: Lifting State & Component Communication

## **Learning Objectives**
By the end of this lesson, students will be able to:
- Understand when and why to lift state up
- Share state between sibling components
- Pass data from child to parent using callback functions
- Manage state in parent components
- Build components that communicate effectively

---

## **Part 1: The Problem - Components Need to Share Data**

### **Scenario: Two Siblings Need the Same Data**

Imagine you have two components that need to share information:

```javascript
// ❌ Problem: Each has its own count
function ComponentA() {
  const [count, setCount] = useState(0);
  return <div>Component A: {count}</div>;
}

function ComponentB() {
  const [count, setCount] = useState(0);
  return <div>Component B: {count}</div>;
}
```

**The Issue:** When Component A updates its count, Component B doesn't know about it. They have separate states!

---

## **Part 2: The Solution - Lifting State Up**

### **What is Lifting State Up?**

**Lifting state up** means moving state to the closest common parent component so that multiple children can share it.

**Rule:** The state should live in the **lowest common ancestor** of all components that need it.

### **How It Works:**

```javascript
// ✅ Solution: Lift state to parent
function Parent() {
  const [count, setCount] = useState(0); // State lives here
  
  return (
    <div>
      <ComponentA count={count} /> {/* Pass as props */}
      <ComponentB count={count} />
      <button onClick={() => setCount(count + 1)}>Increment</button>
    </div>
  );
}

function ComponentA({ count }) {
  return <div>Component A: {count}</div>;
}

function ComponentB({ count }) {
  return <div>Component B: {count}</div>;
}
```

**Now both components display the same count!**

---


## **Part 3: Parent to Child Communication**

This is straightforward - we've been doing this with **props**!

```javascript
function Parent() {
  const [message, setMessage] = useState('Hello from Parent');
  
  return <Child message={message} />;
}

function Child({ message }) {
  return <p>{message}</p>;
}
```

**Data flows DOWN** from parent to child through props.

---


## **Part 4: Child to Parent Communication**

### **The Challenge**

How does a child component send data back to its parent?

**Answer:** The parent passes a **callback function** as a prop, and the child calls it!

### **Example: Child Sends Data to Parent**

```javascript
function Parent() {
  const [name, setName] = useState('');
  
  // This function will be called by the child
  function handleNameChange(newName) {
    setName(newName);
  }
  
  return (
    <div>
      <p>Name in Parent: {name}</p>
      <Child onNameChange={handleNameChange} />
    </div>
  );
}

function Child({ onNameChange }) {
  const [inputValue, setInputValue] = useState('');
  
  function handleSubmit() {
    onNameChange(inputValue); // Call parent's function
  }
  
  return (
    <div>
      <input 
        value={inputValue}
        onChange={(e) => setInputValue(e.target.value)}
      />
      <button onClick={handleSubmit}>Send to Parent</button>
    </div>
  );
}
```

**How it works:**
1. Parent passes `handleNameChange` function to Child
2. Child calls this function when needed
3. Parent's state updates
4. Both components re-render with new data

---


## **Part 5: Sibling Communication**

When two sibling components need to communicate, **lift the state to their parent**.

```javascript
function Parent() {
  const [selectedCity, setSelectedCity] = useState('Lagos');
  
  return (
    <div>
      <CitySelector 
        selectedCity={selectedCity} 
        onSelectCity={setSelectedCity} 
      />
      <CityDisplay city={selectedCity} />
    </div>
  );
}

function CitySelector({ selectedCity, onSelectCity }) {
  return (
    <div>
      <button onClick={() => onSelectCity('Lagos')}>Lagos</button>
      <button onClick={() => onSelectCity('Abuja')}>Abuja</button>
      <button onClick={() => onSelectCity('Port Harcourt')}>Port Harcourt</button>
    </div>
  );
}

function CityDisplay({ city }) {
  return <h2>You selected: {city}</h2>;
}
```

**Data Flow:**
1. CitySelector updates parent's state through callback
2. Parent passes new state to CityDisplay
3. CityDisplay shows updated city

---


## **Part 6: Multiple State Updates**

Sometimes you need to update multiple pieces of state from a child component.

```javascript
function Parent() {
  const [firstName, setFirstName] = useState('');
  const [lastName, setLastName] = useState('');
  const [email, setEmail] = useState('');
  
  function handleFormSubmit(formData) {
    setFirstName(formData.firstName);
    setLastName(formData.lastName);
    setEmail(formData.email);
  }
  
  return (
    <div>
      <h2>User Info:</h2>
      <p>{firstName} {lastName}</p>
      <p>{email}</p>
      
      <UserForm onSubmit={handleFormSubmit} />
    </div>
  );
}

function UserForm({ onSubmit }) {
  const [formData, setFormData] = useState({
    firstName: '',
    lastName: '',
    email: ''
  });
  
  function handleChange(e) {
    setFormData({
      ...formData,
      [e.target.name]: e.target.value
    });
  }
  
  function handleSubmit(e) {
    e.preventDefault();
    onSubmit(formData); // Send all data at once
  }
  
  return (
    <form onSubmit={handleSubmit}>
      <input 
        name="firstName"
        placeholder="First Name"
        value={formData.firstName}
        onChange={handleChange}
      />
      <input 
        name="lastName"
        placeholder="Last Name"
        value={formData.lastName}
        onChange={handleChange}
      />
      <input 
        name="email"
        placeholder="Email"
        value={formData.email}
        onChange={handleChange}
      />
      <button type="submit">Submit</button>
    </form>
  );
}
```

---


## **Part 7: Complete Example - Nigerian Restaurant Order System**

Let's build a complete ordering system where multiple components communicate.

### **App.jsx**

```javascript
import { useState } from 'react';
import MenuList from './MenuList';
import Cart from './Cart';
import './RestaurantApp.css';

function App() {
  const [cartItems, setCartItems] = useState([]);
  
  const menuItems = [
    { id: 1, name: 'Jollof Rice', price: 1500 },
    { id: 2, name: 'Fried Rice', price: 1200 },
    { id: 3, name: 'Pounded Yam & Egusi', price: 2000 },
    { id: 4, name: 'Suya', price: 1000 },
    { id: 5, name: 'Plantain', price: 500 }
  ];
  
  function handleAddToCart(item) {
    const existingItem = cartItems.find(cartItem => cartItem.id === item.id);
    
    if (existingItem) {
      // Increase quantity if item already in cart
      setCartItems(cartItems.map(cartItem =>
        cartItem.id === item.id
          ? { ...cartItem, quantity: cartItem.quantity + 1 }
          : cartItem
      ));
    } else {
      // Add new item to cart
      setCartItems([...cartItems, { ...item, quantity: 1 }]);
    }
  }
  
  function handleRemoveFromCart(itemId) {
    setCartItems(cartItems.filter(item => item.id !== itemId));
  }
  
  function handleUpdateQuantity(itemId, newQuantity) {
    if (newQuantity === 0) {
      handleRemoveFromCart(itemId);
    } else {
      setCartItems(cartItems.map(item =>
        item.id === itemId ? { ...item, quantity: newQuantity } : item
      ));
    }
  }
  
  return (
    <div className="restaurant-app">
      <h1>🍽️ Naija Restaurant</h1>
      <div className="container">
        <MenuList 
          items={menuItems} 
          onAddToCart={handleAddToCart} 
        />
        <Cart 
          items={cartItems}
          onRemove={handleRemoveFromCart}
          onUpdateQuantity={handleUpdateQuantity}
        />
      </div>
    </div>
  );
}

export default App;
```

### **MenuList.jsx**

```javascript
function MenuList({ items, onAddToCart }) {
  return (
    <div className="menu-list">
      <h2>Menu</h2>
      {items.map(item => (
        <div key={item.id} className="menu-item">
          <div>
            <h3>{item.name}</h3>
            <p>₦{item.price.toLocaleString()}</p>
          </div>
          <button onClick={() => onAddToCart(item)}>
            Add to Cart
          </button>
        </div>
      ))}
    </div>
  );
}

export default MenuList;
```

### **Cart.jsx**

```javascript
function Cart({ items, onRemove, onUpdateQuantity }) {
  const total = items.reduce((sum, item) => sum + (item.price * item.quantity), 0);
  
  return (
    <div className="cart">
      <h2>Your Order</h2>
      {items.length === 0 ? (
        <p>Cart is empty</p>
      ) : (
        <>
          {items.map(item => (
            <div key={item.id} className="cart-item">
              <div>
                <h4>{item.name}</h4>
                <p>₦{item.price.toLocaleString()} x {item.quantity}</p>
              </div>
              <div className="cart-actions">
                <button onClick={() => onUpdateQuantity(item.id, item.quantity - 1)}>
                  -
                </button>
                <span>{item.quantity}</span>
                <button onClick={() => onUpdateQuantity(item.id, item.quantity + 1)}>
                  +
                </button>
                <button onClick={() => onRemove(item.id)} className="remove-btn">
                  Remove
                </button>
              </div>
            </div>
          ))}
          <div className="total">
            <h3>Total: ₦{total.toLocaleString()}</h3>
          </div>
        </>
      )}
    </div>
  );
}

export default Cart;
```

### **RestaurantApp.css**

```css
.restaurant-app {
  max-width: 1200px;
  margin: 0 auto;
  padding: 20px;
}

.restaurant-app h1 {
  text-align: center;
  color: #008751;
}

.container {
  display: grid;
  grid-template-columns: 2fr 1fr;
  gap: 20px;
}

.menu-list, .cart {
  background: white;
  padding: 20px;
  border-radius: 10px;
  box-shadow: 0 2px 10px rgba(0,0,0,0.1);
}

.menu-item {
  display: flex;
  justify-content: space-between;
  align-items: center;
  padding: 15px;
  border-bottom: 1px solid #eee;
}

.menu-item button {
  background-color: #008751;
  color: white;
  border: none;
  padding: 8px 16px;
  border-radius: 5px;
  cursor: pointer;
}

.menu-item button:hover {
  background-color: #006741;
}

.cart-item {
  padding: 15px;
  border-bottom: 1px solid #eee;
}

.cart-actions {
  display: flex;
  gap: 10px;
  align-items: center;
  margin-top: 10px;
}

.cart-actions button {
  padding: 5px 10px;
  border: 1px solid #ddd;
  background: white;
  cursor: pointer;
  border-radius: 3px;
}

.remove-btn {
  background-color: #d32f2f !important;
  color: white !important;
  border: none !important;
}

.total {
  margin-top: 20px;
  padding-top: 20px;
  border-top: 2px solid #008751;
  text-align: right;
}
```

### **What We Used:**
✅ **Lifted state**: `cartItems` lives in parent (App)  
✅ **Parent to child**: Passing menuItems and cartItems as props  
✅ **Child to parent**: Callbacks (`onAddToCart`, `onRemove`, `onUpdateQuantity`)  
✅ **Sibling communication**: MenuList and Cart communicate through parent  
✅ **Complex state updates**: Adding, removing, updating quantities  

---


## **Part 8: Tasks**

## 🎯 Task 1: Temperature Converter

**Task:** Build a temperature converter where Celsius and Fahrenheit inputs stay in sync

**Requirements:**
1. Create parent component `TemperatureApp.jsx`
2. Create two child components: `CelsiusInput.jsx` and `FahrenheitInput.jsx`
3. Lift temperature state to parent
4. When user types in Celsius, Fahrenheit updates automatically
5. When user types in Fahrenheit, Celsius updates automatically
6. Formula: F = (C × 9/5) + 32, C = (F - 32) × 5/9

**Sample Output:**
```
Temperature Converter

Celsius: [25]
Fahrenheit: [77]
```

## 🎯 Task 2: Color Picker

**Task:** Create a color picker where RGB sliders update a color preview

**Requirements:**
1. Create parent component `ColorPicker.jsx`
2. Create `RGBSlider.jsx` component (reusable for R, G, B)
3. Create `ColorDisplay.jsx` component to show the color
4. Lift RGB values state to parent
5. Each slider ranges from 0-255
6. Display the RGB values and hex code
7. Show color preview box

**Sample Output:**
```
🎨 Color Picker

Red: [255] ━━━━━━●
Green: [100] ━━●━━━━
Blue: [50] ━●━━━━━━

RGB: (255, 100, 50)
HEX: #FF6432
[Color Preview Box]
```

**Challenge:** Add a "Copy Hex" button that copies the hex code


## 🎯 Task 3: Todo List with Filter

**Task:** Build a todo list where adding/filtering is split between components

**Requirements:**
1. Create parent `TodoApp.jsx`
2. Create `TodoInput.jsx` - input field and add button
3. Create `TodoList.jsx` - displays todos
4. Create `TodoFilter.jsx` - buttons to filter (All, Active, Completed)
5. Lift todos state and filter state to parent
6. Each todo has: text, id, completed status
7. Implement add, toggle complete, and delete functionality

**Sample Output:**
```
📝 My Todo List

[Add new todo...] [Add]

Filter: [All] [Active] [Completed]

☑ Learn React (completed)
☐ Build Projects
☐ Get a Job

Total: 3 tasks | Completed: 1
```

**Challenge:** Add "Clear Completed" button


## 🎯 Task 4: Nigerian States Quiz

**Task:** Build a quiz app where question display and answer checking are separate

**Requirements:**
1. Create parent `QuizApp.jsx`
2. Create `Question.jsx` - displays current question and options
3. Create `ScoreBoard.jsx` - shows current score
4. Create `AnswerFeedback.jsx` - shows if answer was correct/wrong
5. Have 5 questions about Nigerian states and capitals
6. Lift quiz state to parent (current question, score, answered)
7. When user selects answer, show feedback and move to next question
8. Show final score at the end

**Sample Output:**
```
Nigerian States Quiz
Score: 2/5

Question 3:
What is the capital of Lagos State?

○ Lagos Island
○ Ikeja
○ Victoria Island
○ Lekki

[Submit Answer]
```

**Challenge:** Add a "Restart Quiz" button at the end

---

## **Part 9: Review**

### **Key Takeaways**
✅ Lift state to the closest common parent when siblings need to share data  
✅ Data flows down through props (parent → child)  
✅ Events flow up through callbacks (child → parent)  
✅ One source of truth - state should live in one place  
✅ Pass functions as props to let children update parent state  

### **Common Mistakes**
- Not lifting state high enough (siblings can't communicate)
- Lifting state too high (unnecessarily complex)
- Forgetting to pass callback functions to children
- Trying to directly modify parent state from child
- Not using proper function naming (use `onSomething` for event handlers)

### **Best Practices**
- Name callback props starting with `on` (onSubmit, onChange, onAddToCart)
- Keep state as local as possible, lift only when needed
- Use clear function names that describe what they do
- Pass only necessary props to children

---

## **Next Lesson Preview**
Tomorrow we'll learn about **Lists & Conditional Rendering** - how to display dynamic data and show/hide components based on conditions!